In [101]:
import torch

In [102]:
x=torch.tensor(3.0,requires_grad=True)
x

tensor(3., requires_grad=True)

In [103]:
y=x**2
y

tensor(9., grad_fn=<PowBackward0>)

In [104]:
y.backward()

In [105]:
deri=x.grad
deri

tensor(6.)

In [106]:
w=torch.tensor(1.0)
b=torch.tensor(0.0)
x=torch.tensor(6.7)
y=torch.tensor(0.0)

In [107]:
z=w*x+b

In [108]:
y_pred=torch.sigmoid(z)

In [109]:
y_pred

tensor(0.9988)

In [110]:
loss=-torch.log(torch.tensor(1-0.99877))
loss

tensor(6.7007)

In [111]:
dl_dw=(-(y/y_pred - (1-y)/(1-y_pred)))*torch.sigmoid(z)*(1-torch.sigmoid(z))*x
dl_db=(-(y/y_pred - (1-y)/(1-y_pred)))*torch.sigmoid(z)*(1-torch.sigmoid(z))*1
print("dl/dw : ",dl_dw)
print("dl/db : ",dl_db)

dl/dw :  tensor(6.6918)
dl/db :  tensor(0.9988)


In [112]:
w=torch.tensor(1.0,requires_grad=True)
b=torch.tensor(0.0,requires_grad=True)
x=torch.tensor(6.7)
y=torch.tensor(0.0)

In [113]:
def bcel(y,y_pred):
    return -(y*torch.log(y_pred)+(1-y)*torch.log(1-y_pred))

In [114]:
z=w*x+b
y_pred=torch.sigmoid(z)
loss=bcel(y,y_pred)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [115]:
loss.backward()

In [116]:
print("w_grad : ",w.grad)
print("b_grad : ",b.grad)

w_grad :  tensor(6.6918)
b_grad :  tensor(0.9988)


eg:**2**

In [117]:
x=torch.tensor([2,-5,3],dtype=float)
w=torch.tensor([2,1.2,2.1],requires_grad=True,dtype=float)
b=torch.tensor(-1,requires_grad=True,dtype=float)
y=(x**2).mean()

In [118]:

z=torch.dot(w,x)+b
y_pred=torch.sigmoid(z)
loss=bcel(y,y_pred)
print(z)
print(y_pred)
print(loss)

tensor(3.3000, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(0.9644, dtype=torch.float64, grad_fn=<SigmoidBackward0>)
tensor(-38.4638, dtype=torch.float64, grad_fn=<NegBackward0>)


In [119]:

loss.backward()

dl_dw=w.grad
dl_db=b.grad

print("w_grad : ",dl_dw)
print("b_grad : ",dl_db)

w_grad :  tensor([-23.4045,  58.5112, -35.1067], dtype=torch.float64)
b_grad :  tensor(-11.7022, dtype=torch.float64)


In [120]:
print(w)
print(b)

tensor([2.0000, 1.2000, 2.1000], dtype=torch.float64, requires_grad=True)
tensor(-1., dtype=torch.float64, requires_grad=True)


In [121]:
epochs=1000
lr=10

x=torch.tensor([2,-5,3],dtype=float)
w=torch.tensor([2,1.2,2.1],requires_grad=True,dtype=float)
b=torch.tensor(2,requires_grad=True,dtype=float)
#y = torch.tensor(1.0, dtype=torch.float32)
y=(x**2).mean()

for i in range(100):
  z=torch.dot(w,x)+b
  y_pred=torch.sigmoid(z)

  #loss=torch.binary_cross_entropy_with_logits(z, y)
  loss=(y_pred - y)**2

  loss.backward()

  with torch.no_grad():
    w-=lr*w.grad
    b-=lr*b.grad
    w.grad.zero_()
    b.grad.zero_()
  if(i%1000==0): print("loss : ",loss)

loss :  tensor(136.1539, dtype=torch.float64, grad_fn=<PowBackward0>)


#  #4

In [122]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [123]:
# breast cancel dataset from github

df=pd.read_csv('http://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [124]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [125]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

In [126]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [127]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [128]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)

In [129]:
X_train_tensor=torch.from_numpy(X_train)
X_test_tensor=torch.from_numpy(X_test)
y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

In [130]:
w=X_train_tensor.shape[1]

In [131]:
class myNN:
  def __init__(self,X):
    self.w=torch.rand(X.shape[1],dtype=torch.float64,requires_grad=True)
    self.b=torch.zeros(1,dtype=torch.float64,requires_grad=True)

  def forward(self,X):
    z=torch.matmul(X,self.w)+self.b
    y_pred=torch.sigmoid(z)
    return y_pred

  def loss(self,y_pred,y):
    epsilon=1e-7
    y_pred=torch.clamp(y_pred,epsilon,1-epsilon)
    loss=-(y*torch.log(y_pred)+(1-y)*torch.log(1-y_pred))
    return loss.mean()

In [132]:
lr=0.1
epochs=1000

model=myNN(X_train_tensor)

for i in range(epochs):
  y_pred=model.forward(X_train_tensor)
  loss=model.loss(y_pred,y_train_tensor)

  loss.backward()

  with torch.no_grad():
    model.w-=lr*model.w.grad
    model.b-=lr*model.b.grad

  model.w.grad.zero_()
  model.b.grad.zero_()

  if(i%100==0): print(f"epoch : {i+1},   loss : {loss}")

epoch : 1,   loss : 0.5929900664044763
epoch : 101,   loss : 0.0967655596333268
epoch : 201,   loss : 0.08343264218556898
epoch : 301,   loss : 0.07719580601001894
epoch : 401,   loss : 0.0730147117915952
epoch : 501,   loss : 0.06993461258674823
epoch : 601,   loss : 0.06753222708816513
epoch : 701,   loss : 0.06558111288853367
epoch : 801,   loss : 0.0639487918029
epoch : 901,   loss : 0.06255242958569465


In [133]:
from sklearn.metrics import accuracy_score

y_pred=model.forward(X_test_tensor)
y_pred=y_pred.detach().numpy()
y_pred=np.where(y_pred>0.5,1,0)


print(f"accuracy : {accuracy_score(y_pred,y_test_tensor)}")

accuracy : 0.9824561403508771


#   #5

In [146]:
import torch.nn as nn

class model(nn.Module):

  def __init__(self,nn_features):
    super(model,self).__init__()
    self.network=nn.Sequential(
        nn.Linear(nn_features,3),
        nn.ReLU(),
        nn.Linear(3,1),
        nn.Sigmoid()
    )

  def forward(self,x):
    x=self.network(x)
    return x

  def loss(self,y_pred,y):
    return nn.BCEWithLogitsLoss(y_pred,y)


In [135]:
features=torch.rand(10,5)

apun=model(features.shape[1])

apun.forward(features)

tensor([[0.3706],
        [0.3707],
        [0.3871],
        [0.4311],
        [0.4092],
        [0.3945],
        [0.4029],
        [0.4441],
        [0.3869],
        [0.3747]], grad_fn=<SigmoidBackward0>)

In [ ]:
from torchinfo import summary
summary(apun,input_size=(10,5))

In [160]:
X_train_tensor=X_train_tensor.to(torch.float32)
y_train_tensor=y_train_tensor.to(torch.float32)
X_test_tensor=X_test_tensor.to(torch.float32)
y_test_tensor=y_test_tensor.to(torch.float32)

In [184]:
import torch.optim as optim

In [179]:
class myNN(nn.Module):
  def __init__(self,l1_features):
    super(myNN,self).__init__()
    self.linear=nn.Linear(l1_features,1)
    self.criterion = nn.BCEWithLogitsLoss()

  def forward(self,x):
    x=self.linear(x)
    return x

  def loss(self,y_pred,y):
    return self.criterion(y_pred,y)

In [189]:
lr=0.1
epochs=1000

model=myNN(X_train_tensor.shape[1])

optimizer=optim.SGD(model.parameters(),lr=lr)

for i in range(epochs):
  y_pred=model.forward(X_train_tensor)
  loss=model.loss(y_pred,y_train_tensor.view(-1,1))

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  if(i%100==0): print(f"epoch : {i+1},   loss : {loss}")

epoch : 1,   loss : 0.6062687635421753
epoch : 101,   loss : 0.10663679242134094
epoch : 201,   loss : 0.08739578723907471
epoch : 301,   loss : 0.07891137897968292
epoch : 401,   loss : 0.07380996644496918
epoch : 501,   loss : 0.07027716189622879
epoch : 601,   loss : 0.06762513518333435
epoch : 701,   loss : 0.06552878767251968
epoch : 801,   loss : 0.06381161510944366
epoch : 901,   loss : 0.062368299812078476
